In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import datetime
import math
from ipywidgets import interact,widgets

plt.rcParams['figure.figsize'] = [20, 10]

In [43]:
def predictcases(days_to_predict,t_lastknown,t_int,lastnumberofcases,beta0,beta1,K,gamma):

    #create an array for days to be predicted
    predicted_days = range(t_lastknown+1,t_lastknown+1+days_to_predict)

    #initialize predicted cases
    predicted_cases = np.zeros((days_to_predict,))
    
    #initialize daily cases
    predicted_daily_cases = np.zeros((days_to_predict,))

    #intialize beta
    beta = 0

    #fill predicted_cases using discrete time time dependent SIR model
    #Assumption S = N
    for d in predicted_days:

        beta = beta1+(beta0-beta1)*math.exp(-(d-t_int)/K)

        if d == t_lastknown+1:

            predicted_cases[d-t_lastknown-1] = (beta+1)*lastnumberofcases
            predicted_daily_cases[d-t_lastknown-1] = beta*lastnumberofcases

        else:

            predicted_cases[d-t_lastknown-1] = (beta+1)*predicted_cases[d-t_lastknown-2]
            predicted_daily_cases[d-t_lastknown-1] = beta*predicted_cases[d-t_lastknown-2]

    return predicted_daily_cases,predicted_cases,predicted_days

In [44]:
def fitbeta(beta_vector,dates_vector,lockdowndate):

    #intialize the intervention time
    t_int = 0

    #find the intervention time
    for t in range(dates_vector.shape[0]):

        if dates_vector[t] == dates.datestr2num(lockdowndate):

            t_int = t

    #initialize K_errors
    K_errors = np.zeros((200,))
    beta0_records = np.zeros(K_errors.shape)
    beta1_records = np.zeros(K_errors.shape)

    for K in range(1,201):

        #initialize beta0, beta1, error, error_prev
        beta0 = np.mean(beta_vector)
        beta1 = np.mean(beta_vector)
        error = 0
        error_prev = 10000

        #stop the estimation of beta0 and beta1 if the error is not improving
        while(error_prev-error>10**-5 and beta0 > 0 and beta1 > 0):

        #for iters in range(100):


            error_prev = error

            #calculate beta1 based on beta0
            beta1_num = 0
            beta1_den = 0

            for t in range(t_int+1,dates_vector.shape[0]):

                beta1_num += (1-math.exp(-(t-t_int)/K))*beta_vector[t] + beta0*math.exp(-(t-t_int)/K)*(math.exp(-(t-t_int)/K)-1)
                beta1_den += (1-math.exp(-(t-t_int)/K))**2

            beta1 = beta1_num/beta1_den

            #use this beta1 value to estimate beta0

            A = 0
            B = 0
            C = 0

            for t in range(t_int+1):

                A+=beta_vector[t]

            for t in range(t_int+1,dates_vector.shape[0]):

                B+=math.exp(-(t-t_int)/K)*(beta_vector[t]-beta1+beta1*math.exp(-(t-t_int)/K))
                C+=math.exp(-2*(t-t_int)/K)

            beta_0 = (A+B)/(t_int+1+C)

            #beta should be non-negative

            if beta0 < 0:
                beta0 = 0
            if beta1 < 0:
                beta1 = 0


            #calculate the error value

            error = 0

            for t in range(dates_vector.shape[0]):

                if t<t_int+1:

                    error+=(beta_vector[t]-beta0)**2

                else:

                    error+=(beta_vector[t]-beta1-(beta0-beta1)*math.exp(-(t-t_int)/K))**2

        #after the error value is set, record it in order to select the best K later
        K_errors[K-1] = error
        beta0_records[K-1] = beta0
        beta1_records[K-1] = beta1

    #find the K value which gives the minimum array
    index = np.where(K_errors == np.amin(K_errors))
    K = index[0]+1
    beta0 = beta0_records[index]
    beta1 = beta1_records[index]

    return beta0,beta1,K

In [45]:
def run(county_name,days_to_predict,stateData):
    
    if (sum(illinois_data['county'].isin([county_name]))==0):
    
        print('Unfortunately, there is no county called %s in Illinois.' %(county_name))

    else: #from here on, the code runs under this else
    
        county_data = illinois_data[illinois_data['county'].isin([county_name])]

        #get county cases
        cases_raw = county_data['cases']
        cases = cases_raw.values

        #get dates
        county_dates_raw = county_data['date']
        county_dates = dates.date2num(county_dates_raw)

        #crop the data before a certain date

        cropdate = pd.to_datetime('2020-03-15') #'yyyy-mm-dd'
        cases = cases[county_dates_raw>cropdate]
        county_dates = county_dates[county_dates_raw>cropdate]

        #set the lockdown date
        lockdowndate = '2020-03-20'

        #calculate the exact beta for each day

        beta_exact = np.zeros(cases.shape)

        for d in range(cases.shape[0]):

            if d < cases.shape[0]-1:
                beta_exact[d] = (cases[d+1]-cases[d])/cases[d]
            else:
                beta_exact[d] = beta_exact[d-1]

        #fit parameters

        beta0,beta1,K = fitbeta(beta_exact,county_dates,lockdowndate)

        print('beta_0 = %f beta_1 = %f kappa = %f' %(beta0,beta1,K))
        
        #find the intervention time
        for t in range(county_dates.shape[0]):

            if county_dates[t] == dates.datestr2num(lockdowndate):

                t_int = t

        days_num = range(county_dates.shape[0])
        t_smooth = np.linspace(0,county_dates.shape[0],num = 1000)
        beta_smooth = np.zeros(t_smooth.shape)

        for i in range(t_smooth.shape[0]):

            if t_smooth[i] <= t_int:
                beta_smooth[i] = beta0
            else:
                beta_smooth[i] = beta1+(beta0-beta1)*math.exp(-(t_smooth[i]-t_int)/K)

        plt.figure(1)
        plt.plot(days_num,beta_exact,'b*')
        plt.plot(t_smooth,beta_smooth,'--b')
        plt.title('%s County' %(county_name))
        plt.xlabel('date')
        plt.ylabel('$\\beta$')
        plt.show()
        
        daily_predicted,cases_predicted,predicted_days = predictcases(days_to_predict,days_num[-1], \
t_int,cases[-1],beta0,beta1,K,gamma)
        
        #get newly confirmed cases for each day at each county

        daily = np.diff(cases)
        base_date = pd.to_datetime('2020-03-16') #one day after the cropdate
        date_list = [base_date + datetime.timedelta(days=x) for x in range(daily.shape[0])]
        date_list_predicted = [date_list[-1] + datetime.timedelta(days=x+1) for x in range(daily_predicted.shape[0])]

        #convert dates to matplotlib dates
        date_list = dates.date2num(date_list)
        date_list_predicted = dates.date2num(date_list_predicted)
        
        #plot new cases each day
        plt.figure(11)

        ax = plt.subplot(111)
        ax.bar(date_list,daily,width=1,color = 'blue')
        ax.bar(date_list_predicted,daily_predicted,width=1,color = 'orange')
        ax.xaxis_date()
        plt.show()
        

In [46]:
#calculate an estimate of gamma using Hubei province data from JHU dataset

#import data
datajhu = pd.read_csv('jhuyagiz.csv',parse_dates = ['date'])

#get Hubei data
hubei_data = datajhu[datajhu['location'].isin(['Hubei'])]

#get Hubei cases
hubei_cases_raw = hubei_data['cases']
hubei_cases = hubei_cases_raw.values

#get Hubei recovered
hubei_recovered_raw = hubei_data['recovered']
hubei_recovered = hubei_recovered_raw.values

#create days array
hubei_days = range(hubei_cases.shape[0])
hubei_beta = np.zeros(hubei_cases.shape[0])
hubei_gamma = np.zeros(hubei_recovered.shape[0])

for d in hubei_days:
    if d < hubei_days[-1]:
        hubei_gamma[d] = (hubei_recovered[d+1]-hubei_recovered[d])/hubei_cases[d]
    else:
        hubei_gamma[d] = hubei_gamma[d-1]

gamma = np.amax(hubei_gamma)
print('gamma = %f' %(gamma))

#########################################################################

gamma = 0.054547


In [47]:
#import counties data
data = pd.read_csv('us-counties.csv',parse_dates = ['date'])

print(data)

             date      county       state     fips  cases  deaths
0      2020-01-21   Snohomish  Washington  53061.0      1       0
1      2020-01-22   Snohomish  Washington  53061.0      1       0
2      2020-01-23   Snohomish  Washington  53061.0      1       0
3      2020-01-24        Cook    Illinois  17031.0      1       0
4      2020-01-24   Snohomish  Washington  53061.0      1       0
...           ...         ...         ...      ...    ...     ...
179691 2020-05-26    Sublette     Wyoming  56035.0      3       0
179692 2020-05-26  Sweetwater     Wyoming  56037.0     25       0
179693 2020-05-26       Teton     Wyoming  56039.0    100       1
179694 2020-05-26       Uinta     Wyoming  56041.0     12       0
179695 2020-05-26    Washakie     Wyoming  56043.0     32       2

[179696 rows x 6 columns]


In [48]:
#get Illinois Data

illinois_data = data[data['state'].isin(['Illinois'])]

In [49]:
interact(lambda county_name, days_to_predict: run(county_name,days_to_predict,illinois_data),county_name = widgets.Text(value = 'Cook'), days_to_predict = widgets.IntSlider(value = 15, min = 1, max = 50, step = 1, description = 'Days: '))

interactive(children=(Text(value='Cook', description='county_name'), IntSlider(value=15, description='Days: ',…

<function __main__.<lambda>(county_name, days_to_predict)>